# Numerical Simulation of Ballistic Performance of Honeycomb Sandwich Panels

### Objective:
Setup a parametric model in PyMAPDL for a honeycomb sandwich panel. Use it for running a Ballistic impact simulation

The learning objectives of this demo are:
* Setup and solve a parametric model using PyMAPDL
* Interactive plotting of CAD, mesh, and results in Pythonic interface.

### Model Details

#### Model parameters:
* Cell size
* Cell wall thickness
* Node length
* Facesheet thickness

## Step 1 - define all parameters

In [1]:
## Main parameters
cell_size = 0.006
cell_wall_thickness = 0.002
node_length = 0.004
facesheet_thickness = 0.005

In [2]:
# Fixed constants
structure_length = 0.1
structure_breadth = 0.1

In [3]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Additional calculation 
L = cell_size/ np.sqrt(3)
offset_x = cell_size + cell_wall_thickness
offset_y = 1.5*L + cell_wall_thickness*(np.sin(np.pi/3))
Nx = int(structure_length / offset_x) + 1
Ny = int(structure_breadth / offset_y) + 1
print("L = ", L)
print("offset_x = ", offset_x)
print("offset_y = ", offset_y)
print("Nx = ", Nx)
print("Ny = ", Ny)

L =  0.0034641016151377548
offset_x =  0.008
offset_y =  0.0069282032302755096
Nx =  13
Ny =  15


# Step 2 - launch MAPDL and create geometry

In [4]:
from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()
print(mapdl)

LicenseServerConnectionError: 2023/10/23 15:03:15    DENIED              ansys                           23.1 (2022.1114)             1/0/0/0                 1/1/1/1   12324:FEAT_ANSYS:HPRVa@Vamsi:winx64                        7436:10.250.8.137   
		Request name ansys does not exist in the licensing pool.
		The desired vendor daemon is down.
		 Check the lmgrd log file, or try lmreread.
		Feature:       ansys
		Vendor:Host:   localhost
		License path:  1055@localhost;
		FlexNet Licensing error:-97,121

2023/10/23 15:03:15    INFO                Unable to connect to FLEXlm path 1055@localhost; adding to "Unavailable FLEXlm Servers" cache.
2023/10/23 15:03:15    DENIED              mech_2                          23.1 (2022.1114)             1/0/0/0                 1/1/1/1   12324:FEAT_ANSYS:HPRVa@Vamsi:winx64                        7436:10.250.8.137   
		Request name mech_2 does not exist in the licensing pool.
		The desired vendor daemon is down.
		 Check the lmgrd log file, or try lmr

In [ ]:
## reset mapdl
mapdl.clear()

## enter the preprocessor
mapdl.prep7()

mapdl.units('SI')      # SI unit system

## Create geometry
mapdl.block(0, structure_length, 0, structure_breadth, 0, node_length)
for i in range(0,Nx):                                                                                   ## column number
    for j in range(0,Ny):
        if j%2 == 0:                                                                                    ## even row number
            mapdl.rpr4(6, i*offset_x , j*offset_y, L, 90, node_length)
        else:                                                                                           ## odd row number
            mapdl.rpr4(6, i*offset_x + offset_x/2 , j*offset_y, L, 90, node_length)
mapdl.vsbv(1,'all')
honeycomb_core = mapdl.cm("CORE", "VOLU")

front_facesheet = mapdl.block(0, structure_length, 0, structure_breadth, node_length, node_length + facesheet_thickness)  ## Top facesheet
rear_facesheet = mapdl.block(0, structure_length, 0, structure_breadth, -1*facesheet_thickness, 0)                        ## Bottom facesheet

mapdl.clocal( 11, 0, structure_length/2, structure_breadth/2, 0)                                         ## Making local coordinate system
mapdl.wpcsys('', 11,)                                                                                    ## Shift working plane

initial_distance = node_length + facesheet_thickness + 0.02                                              ## 20 mm from center of front facesheet of sandwich panel

# 0.44 Remington Magnum Bullet
mapdl.cone(0.00545, 0.005486, initial_distance , initial_distance + 0.00825)
mapdl.cone(0.00569, 0.005805, initial_distance + 0.00825 , initial_distance + 0.04089 )
mapdl.vadd(3,4)
bullet = mapdl.cm("BULLET","VOLU")

# mapdl.input('bullet9.anf')
# mapdl.finish()

mapdl.vplot('all')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 3 - define material properties, mesh attributes and generate mesh

In [ ]:
## Define material properties
mapdl.mp("EX", 1, 2e5)  # Youngs modulus
mapdl.mp("PRXY", 1, 0.3367)  # Poissons ratio

## Define element attributes
mapdl.et(1, "SOLID185") # 3D 8-Node Layered Solid

## Define mesh controls

mapdl.lesize("ALL", 0.001, layer1=1)
# mapdl.smrtsize(1)

# Mesh facesheets
mapdl.mshape(0, "3D")    # mesh the volume with 3D Hexahedral elements
mapdl.mshkey(1)          # mapped mesh
mapdl.vmesh('2')
mapdl.mshape(0, "3D")    # mesh the volume with 3D Hexahedral elements
mapdl.mshkey(1)          # mapped mesh
mapdl.vmesh('1')

# Mesh honeycomb core
mapdl.mshape(1, "3D")    # mesh the volume with 3D Tetrahedral elements
mapdl.mshkey(0)          # free mesh
mapdl.vmesh('CORE')

# Mesh bullet
mapdl.mshape(1, "3D")    # mesh the volume with 3D Tetrahedral elements
mapdl.mshkey(0)          # free mesh
mapdl.vmesh('BULLET')

## Plot the mesh
mapdl.eplot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 4 - apply loads and boundary conditions

In [ ]:
### Make the lateral sides of the panel as fixed support i.e. set displacement of nodes to zero

# Select all nodes at the x = 0 end of the sandwich panel
mapdl.nsel("S", "LOC", "X", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the x = structure_length end of the sandwich panel
mapdl.nsel("S", "LOC", "X", structure_length)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the y = 0 end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the y = structure_breadth end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", structure_breadth)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")


SPECIFIED CONSTRAINT UX   FOR SELECTED NODES            1 TO      181249 BY           1
 REAL=  0.00000000       IMAG=  0.00000000    
 ADDITIONAL DOFS=  UY    UZ

In [ ]:
mapdl.exit()